In [64]:
from unstructured.partition.pdf import partition_pdf
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain_ollama import OllamaLLM
from langchain.schema.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore, LocalFileStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from langchain_ollama.chat_models import ChatOllama
from langchain_groq.chat_models import ChatGroq
from unstructured.partition.text import partition_text
import os
import sys
import uuid
from PIL import Image
import pytesseract

In [65]:
model_deepseek = OllamaLLM(
    model = "deepseek-r1:8b",
    base_url = "127.0.0.1:11434"
)

model_gemma = OllamaLLM(
    model = "gemma3:4b",
    base_url = "127.0.0.1:11434"
)

chat_model = ChatOllama(
    model = "gemma3:4b",
    base_url = "http://127.0.0.1:11434"
)

# chat_groq = ChatGroq(
#     model = "gemma2-9b-it",
#     api_key = "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"
# )

chat_groq = ChatGroq(
    model = "gemma2-9b-it",
    api_key = "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"
)

embedding = OllamaEmbeddings(model = "nomic-embed-text")

DATA_FOLDER = "data"

In [127]:
res = chat_model.invoke("what color is sky?")
print(res)

content="That's a fantastic and surprisingly complex question! The sky isn’t actually a single color. Here's the breakdown of why it appears blue (most of the time):\n\n* **Rayleigh Scattering:** The primary reason the sky appears blue is due to a phenomenon called Rayleigh scattering. Sunlight is made up of all the colors of the rainbow. As sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen). This scattering of light is more effective with shorter wavelengths like blue and violet. Blue light is scattered more than other colors, so it gets bounced around in all directions.  This scattered blue light is what we see when we look up.\n\n* **Why Not Violet?** Violet light is actually scattered even *more* than blue light. However, several factors contribute to why we see blue instead of violet:\n    * **Sunlight’s Composition:** The sun emits slightly less violet light than blue light.\n    * **Our Eyes’ Sensitivity:** Our eyes are less 

In [ ]:
def process_pdf():
    files = os.listdir(r'/home/shiva/Downloads/data_folder')
    print(files)
    for f in files:
        if f.lower().endswith('.pdf'):
            chunk = partition_pdf(
                filename = os.path.join(r"/home/shiva/Downloads/data_folder", f),
                infer_table_structure=True, 
                strategy="hi_res",
                extract_image_block_types=["Image"],
                extract_image_block_to_payload=True, 
                chunking_strategy="by_title",         
                max_characters=10000,                  
                combine_text_under_n_chars=2000,       
                new_after_n_chars=6000,
            )
    
    return chunk

In [82]:
pdf_chunks = process_pdf()

['attentio.pdf', 'pyq.jpeg', 'OIP.webp']


In [43]:
chunk = partition_text(
                    filename = os.path.join(r'/home/shiva/Downloads/data_folder', 'Prehistoric medicine Main.txt'),
                    encoding = "utf-8",
                    max_characters=10000,
                    combine_text_under_n_chars=2000,
                    new_after_n_chars=6000,
                    chunking_strategy= "by_title"
                )

In [57]:
for i in chunk:
    print(i)

Prehistoric medicine Main article: Prehistoric medicine Yarrow, a medicinal plant found in human-occupied caves in the Upper Palaeolithic period[3]

Prehistoric medicine is a field of study focused on understanding the use of medicinal plants, healing practices, illnesses, and wellness of humans before written records existed.[4] Although styled prehistoric "medicine", prehistoric healthcare practices were vastly different from what we understand medicine to be in the present era and more accurately refers to studies and exploration of early healing practices.

This period extends across the first use of stone tools by early humans c. 3.3 million years ago[5] to the beginning of writing systems and subsequent recorded history c. 5000 years ago.

As human populations were once scattered across the world, forming isolated communities and cultures that sporadically interacted, a range of archaeological periods have been developed to account for the differing contexts of technology, socioc

In [49]:
print(chunk[1].metadata.orig_elements)

[<unstructured.documents.elements.NarrativeText object at 0x7ee870e006d0>, <unstructured.documents.elements.NarrativeText object at 0x7ee870f4c640>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd1210>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd2e60>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd0550>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd2260>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd3fd0>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd19c0>, <unstructured.documents.elements.NarrativeText object at 0x7ee870dd2a70>]


In [95]:
([str(type(el)) for el in pdf_chunks])

["<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>",
 "<class 'unstructured.documents.elements.CompositeElement'>"]

In [112]:
# for i in pdf_chunks:
#     print(str(type(i)))

pdf_chunks[3].metadata.orig_elements
# chunk_images = [el for el in pdf_chunks[3].metadata.orig_elements if 'Image' in str(type(el))]
# chunk_images[0].to_dict()

for chunk in pdf_chunks:
    print(chunk.to_dict())

{'type': 'CompositeElement', 'element_id': '8a4e06b7362865934ea57960c1ee753d', 'text': '3\n\n2023\n\n2\n\n0\n\n2\n\ng u A 2 ] L C . s c [ 7 v 2 6 7 3 0 . 6 0 7 1 :\n\nlV: aura\n\nv\n\ni\n\nX\n\nr\n\na\n\nProvided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\nAttention Is All You Need\n\nAshish Vaswani∗\n\nGoogle Brain\n\navaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗\n\nGoogle Research usz@google.com\n\nLlion Jones∗\n\nGoogle Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡\n\nillia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. Th

In [15]:
s1 = pdf_chunks["monopoly.pdf"]
print(s1[0].metadata.orig_elements)

[<unstructured.documents.elements.Title object at 0x737b3ba35fc0>, <unstructured.documents.elements.Image object at 0x737b3ba34c70>, <unstructured.documents.elements.Title object at 0x737bb60f2a40>, <unstructured.documents.elements.NarrativeText object at 0x737b3ba35f60>, <unstructured.documents.elements.NarrativeText object at 0x737bb62c2bc0>, <unstructured.documents.elements.Image object at 0x737b3ba370a0>, <unstructured.documents.elements.NarrativeText object at 0x737bb60618a0>, <unstructured.documents.elements.NarrativeText object at 0x737bb42b56f0>, <unstructured.documents.elements.NarrativeText object at 0x737bb62a8640>, <unstructured.documents.elements.NarrativeText object at 0x737b7d564160>, <unstructured.documents.elements.NarrativeText object at 0x737b7d564850>, <unstructured.documents.elements.Image object at 0x737b3ba37b20>, <unstructured.documents.elements.Title object at 0x737b7d567760>, <unstructured.documents.elements.Title object at 0x737b7d5642b0>, <unstructured.docum

In [7]:
s2 = pdf_chunks["Wasserstoff Gen-AI Internship.pdf"]

In [8]:
for i in s2:
    print(type(i))

print(s1[0].metadata.orig_elements)

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
[<unstructured.documents.elements.Title object at 0x7523eac86fe0>, <unstructured.documents.elements.Image object at 0x7523eac86dd0>, <unstructured.documents.elements.Title object at 0x752461934a30>, <unstructured.documents.elements.NarrativeText object at 0x7523eac86da0>, <unstructured.documents.elements.NarrativeText object at 0x75245f5f8f70>, <unstructured.documents.elements.Image object at 0x7523eac87dc0>, <unstructured.documents.elements.NarrativeText object at 0x752411b39240>, <unstructured.documents.elements.NarrativeText object at 0x7523eaa40910>, <unstructured.documents.elements.NarrativeText object at 0x7523eaae8be0>, <unstructured.documents.elements.NarrativeText object at 0x7524619351e0>, <unstructured.documents.elements.NarrativeText object at 0x7523eac87700>, <unstructured.documents.elements.Image o

In [16]:
text = []
for i in s1:
    if "CompositeElement" in str(type((i))):
        text.append(i)

# for i in s2:
#     if "CompositeElement" in str(type((i))):
#         text.append(i)

In [17]:
text_onl = []
for i in s1:
    if "CompositeElement" in str(type((i))):
        text_onl.append(i.text)

# for i in s2:
#     if "CompositeElement" in str(type((i))):
#         text_onl.append(i.text)

In [18]:
print(sys.getsizeof(text))
# print(sys.getsizeof(text_onl))

184


In [19]:
for i in text_onl:
    print(i)

MONOPOLY Property Trading Game from Parker Brothers" AGES 8+

2 to 8 Players Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance

2 to 8 Players

Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance and Community Chest cards, Title Deed cards, play money and a Banker's tray.

Now there's a faster way to play MONOPOLY. Choose to play by

the classic rules for buying, renting and selling properties or use the Speed Die to get into the action faster. If you've never played the classic If you already know how to play and want to use the Speed Die, just

the classic rules for buying, renting and selling properties or use the

Speed Die to get into the action faster. If you've never played the classic

MONOPOLY game, refer to the Classic Rules beginning on the next page.

If you already know how to play and want to use the Speed Die, just

read the section below for the additional Speed Die rules.

D aes a

a

SPEED DIE RULES

SPEED DIE RULES Learning how to play w

In [20]:
for i in text:
    print(i)

MONOPOLY Property Trading Game from Parker Brothers" AGES 8+

2 to 8 Players Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance

2 to 8 Players

Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance and Community Chest cards, Title Deed cards, play money and a Banker's tray.

Now there's a faster way to play MONOPOLY. Choose to play by

the classic rules for buying, renting and selling properties or use the Speed Die to get into the action faster. If you've never played the classic If you already know how to play and want to use the Speed Die, just

the classic rules for buying, renting and selling properties or use the

Speed Die to get into the action faster. If you've never played the classic

MONOPOLY game, refer to the Classic Rules beginning on the next page.

If you already know how to play and want to use the Speed Die, just

read the section below for the additional Speed Die rules.

D aes a

a

SPEED DIE RULES

SPEED DIE RULES Learning how to play w

In [21]:
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

In [22]:
summarize_chain = {"element": lambda x: x} | prompt | model_gemma | StrOutputParser()
# semmarize_chain = {"element": lambda x: x} | prompt | model_gemma | StrOutputParser()

In [23]:
text_summaries = summarize_chain.batch(text, {"max_concurrency": 3})

In [17]:
# sumamrries_text_only = summarize_chain.batch(text_onl, {"max_concurrency": 3})

In [18]:
# print(sumamrries_text_only)

In [25]:
for i in text_summaries:
    print(i)

The Monopoly Property Trading Game is for 2-8 players aged 8+. It includes a gameboard, dice, tokens, houses, hotels, and Chance/Community Chest cards.  A faster play option, the Speed Die, is available. Players start with an extra $1,000 and use the Speed Die after landing on or passing GO for the first time. The Speed Die allows players to add the roll of the dice to the sum of both dice, or to move the value of one die individually.

Monopoly is a game where players buy properties, pay rent, and aim to become the wealthiest. Players use white dice to move, and if properties remain in the bank, they can buy them or auction them off. If no properties are left, they advance to the next property and owe rent to another player. White dice are only used when rolling for doubles or to get out of jail. Players start with $1,500 and the goal is to acquire property and become the wealthiest player.
The Bank manages the game's finances, holding Title Deed cards, houses, and hotels, paying sala

In [3]:
vector_store = Chroma(collection_name = "storage_vector", embedding_function = embedding)
storage_temp = InMemoryStore()
storage_persistant = LocalFileStore(r"docstore")
id_key = 'doc_id'

retriever = MultiVectorRetriever(
    vectorstore = vector_store,
    docstore = storage_temp,
    id_key = id_key
)

/tmp/ipykernel_50836/947060854.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(collection_name = "storage_vector", embedding_function = embedding)


In [4]:
def clear_db():
    global vector_store, storage_temp, retriever

    try:
        all_vector_ids = vector_store._collection.get()['ids']
        if all_vector_ids:
            vector_store.delete(all_vector_ids)
    except Exception as e:
        print("Error deleting from Chroma:", e)

    vector_store = Chroma(
        collection_name="storage_vector",
        embedding_function=embedding
    )

    try:
        all_doc_ids = list(storage_temp.yield_keys())
        if all_doc_ids:
            storage_temp.mdelete(keys=all_doc_ids)
    except Exception as e:
        print("Error deleting from InMemoryStore:", e)

    storage_temp = InMemoryStore()
    retriever = MultiVectorRetriever(
        vectorstore=vector_store,
        docstore=storage_temp,
        id_key='doc_id'
    )

In [5]:
docs = retriever.invoke("how to win in monopoly game?")

In [6]:
for d in docs:
    print(d)

In [116]:
def summary_pdf(DATA_FOLDER):
    files = os.listdir(DATA_FOLDER)
    processed_file_dict_ids = {}

    for i, f in enumerate(files):
        if f.lower().endswith('.txt'):
            chunk = partition_text(
                filename = os.path.join(DATA_FOLDER, f),
                encoding = "utf-8",
                max_characters=10000,
                combine_text_under_n_chars=2000,
                new_after_n_chars=6000,
                chunking_strategy= "by_title"
            )

            texts = []
            for i in chunk:
                if "CompositeElement" in str(type(ch)):
                    texts.append(i)
            prompt_text = """You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.
Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.
text chunk: {element}
"""
            prompt = ChatPromptTemplate.from_template(prompt_text)
            summarize_chain = {"element": lambda x: x} | prompt | chat_groq | StrOutputParser()
            text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})
            doc_idss = [str(uuid.uuid4()) for _ in texts]
            summary_text = []
            for i, summary in enumerate(text_summaries):
                page_num = texts[i].metadata.page_number
                summary_text.append(Document(
                    page_content=summary,
                    metadata={
                        id_key: doc_idss[i],
                        "file_name": f.lower(),
                        "page_number": page_num
                    }
                ))
            retriever.vectorstore.add_documents(summary_text, ids = doc_idss)
            retriever.docstore.mset(list(zip(doc_idss, texts)))
            processed_file_dict_ids[str(f.lower())] = {
                "txt_docstore_id": doc_idss,
                "txt_vectorstore_id": doc_idss
            }
            
        if f.lower().endswith('.pdf'):
            chunk = partition_pdf(
                filename=os.path.join(DATA_FOLDER, f),
                infer_table_structure=False,
                strategy="hi_res",
                extract_image_block_to_payload=True,
                chunking_strategy="by_title",
                max_characters=10000,
                combine_text_under_n_chars=2000,
                new_after_n_chars=6000,
            )
            texts = []
            for ch in chunk:
                if "CompositeElement" in str(type((ch))):
                    texts.append(ch)
                
            prompt_text = """You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.
Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.
text chunk: {element}
"""
            prompt = ChatPromptTemplate.from_template(prompt_text)
            summarize_chain = {"element": lambda x: x} | prompt | chat_groq | StrOutputParser()
            text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})

            doc_idss = [str(uuid.uuid4()) for _ in texts]
            summary_text = []
            for i, summary in enumerate(text_summaries):
                page_num = texts[i].metadata.page_number
                summary_text.append(Document(
                    page_content=summary,
                    metadata={
                        id_key: doc_idss[i],
                        "file_name": f.lower(),
                        "page_number": page_num
                    }
                ))

            
            retriever.vectorstore.add_documents(summary_text, ids = doc_idss)
            retriever.docstore.mset(list(zip(doc_idss, texts)))
            
            processed_file_dict_ids[str(f.lower())] = {
                "pdf_docstore_id": doc_idss,
                "pdf_vectorstore_id": doc_idss
            }

        if f.lower().endswith('.jpeg') or f.lower().endswith('.png') or f.lower().endswith('.webp'):
            img = Image.open(os.path.join(DATA_FOLDER, f))
            img = img.convert('L')
            text = pytesseract.image_to_string(img)

            if len(text.strip()) > 0:
                with open(f"{f}", "w", encoding="utf-8") as txt_file:
                    txt_file.write(text)
                chunks = partition_text(
                filename= f ,
                chunking_strategy="by_title",
                max_characters=10000,
                combine_text_under_n_chars=2000,
                new_after_n_chars=6000
                )

                texts = []
                for i in chunks:
                    if "CompositeElement" in str(type(ch)):
                        texts.append(i)

                prompt_text = """The text that is given to you is extracted from a image.
If there are some questions asked in the image do not answer them unless told to do so,
just summarize the text, is very short form as you are trying to tell whats in the image.
if questions are present just get a brief of them for future answering purpose,
And at the starting do state that the this is the infomation accoring to the text in the image
text_extracted_from_image = {element}""" 

                prompt = ChatPromptTemplate.from_template(prompt_text)
                summarize_chain = {"element": lambda x: x} | prompt | chat_groq | StrOutputParser()
                text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})
                doc_idss = [str(uuid.uuid4()) for _ in texts]
                summary_text = []
                for i, summary in enumerate(text_summaries):
                    page_num = texts[i].metadata.page_number
                    summary_text.append(Document(
                        page_content=summary,
                        metadata={
                            id_key: doc_idss[i],
                            "file_name": f.lower(),
                            "page_number": page_num
                        }
                    ))
                retriever.vectorstore.add_documents(summary_text, ids = doc_idss)
                retriever.docstore.mset(list(zip(doc_idss, texts)))
                processed_file_dict_ids[str(f)] = {
                    "img_docstore_id": doc_idss,
                    "img_vectorstore_id": doc_idss
                }


    return processed_file_dict_ids

In [117]:
def process_docs_iclude_txt(file_paths, status= None):
    processed_file_dict_ids = {}
    dir_f = os.listdir(file_paths)

    for i, f_name in enumerate(dir_f):
        print(f_name)

        if f_name.lower().endswith('.pdf'):
            chunk = partition_pdf(
                filename=os.path.join(file_paths, f_name),
                infer_table_structure=False,
                strategy="hi_res",
                extract_image_block_to_payload=True,
                chunking_strategy="by_title",
                max_characters=10000,
                combine_text_under_n_chars=2000,
                new_after_n_chars=6000,
            )
            texts = []
            for ch in chunk:
                if "CompositeElement" in str(type((ch))):
                    texts.append(ch)
                
            prompt_text = """
            You are an assistant tasked with summarizing tables and text.
            Give a concise summary of the table or text.

            Respond only with the summary, no additionnal comment.
            Do not start your message by saying "Here is a summary" or anything like that.
            Just give the summary as it is.

            text chunk: {element}

            """
            prompt = ChatPromptTemplate.from_template(prompt_text)
            summarize_chain = {"element": lambda x: x} | prompt | chat_groq | StrOutputParser()
            text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})

            doc_idss = [str(uuid.uuid4()) for _ in texts]
            summary_text = []
            for i, summary in enumerate(text_summaries):
                page_num = texts[i].metadata.page_number
                summary_text.append(Document(
                    page_content=summary,
                    metadata={
                        id_key: doc_idss[i],
                        "file_name": f_name,
                        "page_number": page_num
                    }
                ))

            
            retriever.vectorstore.add_documents(summary_text, ids = doc_idss)
            retriever.docstore.mset(list(zip(doc_idss, summary_text)))
            
            processed_file_dict_ids[str(f_name)] = {
                "pdf_docstore_id": doc_idss,
                "pdf_vectorstore_id": doc_idss
            }

        elif f_name.lower().endswith('.jpeg') or f_name.lower().endswith('.png') or f_name.lower().endswith('.webp'):
            img = Image.open(os.path.join(file_paths, f_name))
            img = img.convert('L')
            text = pytesseract.image_to_string(img)

            if len(text.strip()) > 0:
                chunks = text_splitter.split_text(text) 
            
                prompt_text = """The text that is given to you is extracted from a image.
If there are some questions asked, do not answer them,
just summarize the text, is very short form as you are trying to tell whats in the image.
Do not start the messeage by saying "here is the summary or answer" or anything like that.
And at the starting do state that the summary is of image
text_extracted_from_image = {element}""" 

                prompt = ChatPromptTemplate.from_template(prompt_text)
                summarize_chain = {"element": lambda x: x} | prompt | chat_groq | StrOutputParser()
                img_summaries = summarize_chain.batch(chunks)
                img_summary = " ".join(img_summaries)

                img_id = str(uuid.uuid4()) 
                summary_img = Document(page_content=img_summary, metadata={id_key: img_id, "file_name": f_name, "page_number": None})
                retriever.vectorstore.add_documents([summary_img], ids = [img_id])
                retriever.docstore.mset([(img_id, summary_img)])

                processed_file_dict_ids[str(f_name)] = {
                "img_docstore_id": img_id,
                "img_vectorstore_id": img_id
                }

        if status:
            status["current"] = i + 1

    if status:
        status["processing"] = False

    return processed_file_dict_ids


In [118]:
clear_db()

In [119]:
files_dict = summary_pdf("/home/shiva/Downloads/data_folder")

In [120]:
def modify_learning_file(file_to_be_deleted: list, DATA_FOLDER = "/home/shiva/Downloads/data_folder", files_dict_id = files_dict):

    for i in file_to_be_deleted:
        if i.lower().endswith('.pdf'):
            vector_id = files_dict_id[i]["pdf_docstore_id"]
            vector_store.delete(vector_id)
            storage_temp.mdelete(vector_id)
        elif i.lower().endswith('.txt'):
            vector_id = files_dict_id[i]["txt_docstore_id"]
            vector_store.delete(vector_id)
            storage_temp.mdelete(vector_id)
        else:
            vector_id = files_dict_id[i]["img_docstore_id"]
            vector_store.delete(vector_id)
            storage_temp.mdelete(vector_id)


In [121]:
# modify_learning_file(file_to_be_deleted = ["phto_synth.png", "pyq.jpeg", "monopoly.pdf"])

In [122]:
# print(files_dict["attentio.pdf"])
# print(files_dict["pyq.jpeg"])

In [123]:
entries_vector_store = vector_store._collection.get()['ids']
docstore_entries = list(storage_temp.yield_keys())
print(len(docstore_entries))
print(len(entries_vector_store))

26
26


In [124]:
print(vector_store._collection.get(files_dict["attentio.pdf"]["pdf_docstore_id"]))
print(storage_temp.amget(files_dict["attentio.pdf"]["pdf_docstore_id"]))

{'ids': ['a6269b7b-7c4e-49d7-849f-f0e6e21ccf12', 'a6ae4228-2030-4d50-8124-2df0bef60d60', 'bcd8b305-fad6-4494-9d1c-a8d6f0f60b08', '39c71db6-3c99-496e-b8ed-83696b3b5387', '1045161b-672c-451a-a5fe-fe3587f4a1f8', '2d834ec2-4de6-4350-bff3-879a0cc4743b', 'ee1d11a5-7795-4a34-8fa0-2513d0365829', '8060ccfe-b97b-4a2c-a423-8eb75f960324', '5bdbafdf-a40f-486d-94dc-dcb8c24d5f40', '83518cf2-4d21-4f1a-8012-90f31215d654', '742a695a-a0ca-453a-9f5f-d8cef433a3a7', 'cfebba9b-ebf2-4a4b-bf64-1be180ecd863'], 'embeddings': None, 'documents': ["This paper introduces the Transformer, a novel network architecture for sequence transduction that relies solely on attention mechanisms. Unlike previous models that use recurrent or convolutional neural networks, the Transformer achieves superior performance on machine translation tasks while being more parallelizable and faster to train. It demonstrates state-of-the-art results on both English-to-German and English-to-French translation benchmarks, surpassing previous 

/tmp/ipykernel_50836/3737974493.py:2: RuntimeWarning: coroutine 'InMemoryBaseStore.amget' was never awaited
  print(storage_temp.amget(files_dict["attentio.pdf"]["pdf_docstore_id"]))


In [125]:
print(docstore_entries)
print(entries_vector_store)

['a6269b7b-7c4e-49d7-849f-f0e6e21ccf12', 'a6ae4228-2030-4d50-8124-2df0bef60d60', 'bcd8b305-fad6-4494-9d1c-a8d6f0f60b08', '39c71db6-3c99-496e-b8ed-83696b3b5387', '1045161b-672c-451a-a5fe-fe3587f4a1f8', '2d834ec2-4de6-4350-bff3-879a0cc4743b', 'ee1d11a5-7795-4a34-8fa0-2513d0365829', '8060ccfe-b97b-4a2c-a423-8eb75f960324', '5bdbafdf-a40f-486d-94dc-dcb8c24d5f40', '83518cf2-4d21-4f1a-8012-90f31215d654', '742a695a-a0ca-453a-9f5f-d8cef433a3a7', 'cfebba9b-ebf2-4a4b-bf64-1be180ecd863', '699b994a-3f92-42e7-a3b5-226963da90c1', 'e6cd52c9-4044-4b6c-b6e1-f11f52a01ff8', '317a2e1d-b541-4461-8988-d72237bb52b2', '3e8539c5-2e6e-4f8e-8feb-7cd044ece5d5', 'f56750c1-d30e-4bcc-b559-a9227322590e', '160f8623-f72f-4aa9-87c1-45b0a4fc8a73', '349101e1-7cb9-4b08-b008-706fdeadc68c', 'd16b75f5-4b10-4523-94d8-bc812f7d2d8e', '84b94b38-671b-45ef-87ed-ab76864ac106', '9232e5a7-8877-429e-8c39-eb326f28492d', '1a01fcf9-2fb3-4bde-97f7-4167942757a8', '30a8b943-430e-42ea-9ed5-b6e411ed6781', 'eae2fcfa-4faa-4d4d-b9eb-18a3466c7981',

In [126]:
def parse_docs(docs):
    text = []
    for doc in docs:
        text.append(doc)

    return {"texts": text}

def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

        
    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            print(str(type(text_element)))
            context_text += text_element.text
    
    prompt_template = f"""
    Answer the question based only on the following context, which can include text, tables, and the below image.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )

In [127]:
chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | chat_groq
    | StrOutputParser()
)

chain_with_sources = {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    } | RunnablePassthrough().assign(
        response=(
            RunnableLambda(build_prompt)
            | chat_groq
            | StrOutputParser()
        )
    )

In [128]:
res = chain_with_sources.invoke("what is self attention?")

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>


In [129]:
res3 = chain_with_sources.invoke("what were the medicies being used in prehistoric times?")

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>


In [130]:
res4 = chain_with_sources.invoke("what are the questions given in the image?")

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>


In [131]:
res2 = chain_with_sources.invoke("how do we win in monopoly?")

<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>
<class 'unstructured.documents.elements.CompositeElement'>


In [132]:
print(f"res: {res['response']}")
print(f"res2: {res2['response']}")
print(f"res3: {res3['response']}")
print(f"res4: {res4['response']}")
# for i in res["context"]["texts"]:
#     print(i.text)
#     print(i.metadata.page_number)


res: Self-attention is a mechanism that allows a model to weigh the importance of different parts of an input sequence when processing it.  

Here's a breakdown based on the provided text:

* **How it works:**  It takes a sequence of vectors (representing words or other units) and calculates a "compatibility" score between each pair of vectors. These scores determine how much each vector should influence the output for a given position in the sequence.

* **Key features:**
    * **Parallel computation:** Unlike recurrent models, self-attention can process all positions in the sequence simultaneously, making it more efficient.
    * **Long-range dependencies:**  Self-attention can effectively capture relationships between words that are far apart in the sequence, which is crucial for understanding complex sentences.
    * **Interpretability:**  The attention weights can provide insights into which parts of the input the model is focusing on for a given output.

* **Types:** The text men

In [133]:
for i in res4["context"]["texts"]:
    print(i.metadata.page_number)
    print(i.metadata.filename)
    # for j in i:
    #     print(j.metadata["file_name"])

None
pyq.jpeg
None
phto_synth.png
8
monopoly.pdf
3
attentio.pdf


In [84]:
clear_db()

In [135]:
base, ext = os.path.splitext('path/to/hello.png')
print(base)
print(ext)

path/to/hello
.png
